# The frequency domain

We want to learn about the Fourier transform, and eventually the short-time Fourier transform (STFT) and spectral decomposition.

Let's start in the time domain.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Making waves

### EXERCISE

Implement a function to generate a sine wave signal `s` with amplitude `a` and frequency `f`:

$$ s(t) = a \sin ( 2 \pi f t ) $$

Use `np.linspace` to generate a time series of _even_ length, given (for example) a `duration` of 1 s and a sample interval `dt` of 0.001 s.

In [ ]:
def sine_wave(duration, dt, f, a=1):
    
    # YOUR CODE HERE
    
    return s, t

In [ ]:
def sine_wave(duration, dt, f, a=1):
    """
    Produce a sine wave and return it with its time basis.
    """
    t = np.linspace(0, duration, int(duration/dt))
    return a * np.sin(2 * np.pi * f * t), t

### EXERCISE

Generate and plot a 1-second-long sine wave at 261.63 Hz, sampled at a sampling frequency `fs` of 10 kHz.

In [ ]:
f = 
fs =
s, t = 

assert s.size == 10_000

In [ ]:
fs = 10_000  # Hz sample rate.
s, t = sine_wave(1, 1/fs, f=261.63)

plt.figure(figsize=(15, 2))
plt.plot(t, s)

This frequency corresponds to middle-C, or C4.

In [ ]:
from IPython.display import Audio

Audio(s, rate=fs)

## A chord

Now we want to combine three waves with different frequencies and amplitudes.

In [ ]:
f = np.array([261.6, 329.6, 392.0])  # C4, E4, G4 = C-major
a = np.array([1.5, 0.5, 1])

### EXERCISE

Modify your function to accept arrays for the frequency and amplitude. It should return a signal of shape (3, n), where n is the number of samples.

Recall the 'new axis' trick we used with the `ricker()` function to get it to accept an array of frequencies.

Use your function to generate the three-frequency chord. Make it last 2 seconds at 10 kHz sampling.

In [ ]:
# YOUR CODE HERE
def sine_wave(duration, dt, f, a=1):
    
    # YOUR CODE HERE
    
    return s, t

s_, t =   # Call your function

assert s_.shape == (3, 20_000)

In [ ]:
s_, t = sine_wave(duration=2, dt=1/fs,
                  f=f.reshape(3, 1),
                  a=a.reshape(3, 1),
                 )

In [ ]:
plt.plot(s_.T[:200])

### EXERCISE

Sum the three 'channels' of the signal to get a composite 1D signal. Then:

- Plot it.
- Listen to it.

In [ ]:
# YOUR CODE HERE



In [ ]:
s = np.sum(s_, axis=0)
plt.plot(s[:200])

In [ ]:
Audio(s, rate=fs)

## The Fourier transform

We can obtain the complex spectrum like so:

In [ ]:
S = np.fft.fft(s)

In [ ]:
S

In [ ]:
plt.plot(S)

Notice that the spectrum is complex. The magnitude carries amplitude information; the angle ('argument') part carries phase information.

NumPy gives you the full spectrum of positive and negative frequencies (for the imaginary part of the signal). For real signals, these are the same.

It can be confusing, because the negative portion is tacked onto the end of the positive portion... and it's backwards.

`fftfreq` tells us this:

In [ ]:
np.fft.fftfreq(s.size, d=1/fs)

Instead of trying to juggle everything, I suggest using `fftshift` to put everything where it's supposed to be:

In [ ]:
S_ = np.fft.fftshift(S)

plt.plot(S_)

In [ ]:
freq = np.fft.fftfreq(s.size, d=1/fs)

freq_ = np.fft.fftshift(freq)

In [ ]:
n = s.size / 2  # The spectral magnitude contains energy from the whole signal.
                # It's common to normalize by 1/N, 1/2N or 1/sqrt(N).

In [ ]:
plt.plot(freq_, np.abs(S_)/n)
plt.xlim(0, 500)

## The real spectrum

With most signals, we are only interested in the positive frequencies, so we can use the 'real' FFT, or `rfft()`.

In [ ]:
S = np.fft.rfft(s)
freq = np.fft.rfftfreq(s.size, d=1/fs)

plt.plot(freq, np.abs(S)/n)
plt.xlim(0, 500)

## The phase spectrum

The phase spectrum contains the timing information. 

Instantaneous phase is given by the complex angle:

In [ ]:
plt.plot(freq, np.angle(S))
plt.xlim(0, 500)

## Converting to decibels

It's common to see spectrums plotted on a logarithmic decibel (dB) scale. We can compute this:

$$ \mathrm{dB} = 20 \log_{10} \frac{A}{A_\mathrm{ref}} $$

The amplitude is divided by a 'reference' amplitude, which is usually the maximum amplitude — for this signal, or from your entire collection of signals if you want to compare relative amplitudes.

### EXERCISE

Compute and plot the spectrum `S` in decibels.

In [ ]:
# YOUR CODE HERE



In [ ]:
Aref = np.max(np.abs(S))

dB = 20 * np.log10(np.abs(S) / Aref)

plt.figure(figsize=(15, 3))
plt.plot(freq, dB)

## Apodization

Notice that our spectral peaks are a bit spread out in frequency. This is because it starts and ends abruptly. Abrupt time is spread out in frequency, and vice versa. We can mitigate the effect by **windowing** the signal. For example:

In [ ]:
s_win = np.blackman(s.size) * s

plt.figure(figsize=(15, 2))
plt.plot(s_win)

In [ ]:
S_win = np.fft.rfft(s_win)

plt.plot(freq, np.abs(S)/n)
plt.plot(freq, np.abs(S_win)/n)
plt.xlim(0, 500)

### EXERCISE

Try implementing the `tukey()` window from `scipy.signal`.

What do you notice about the magnitudes of the Fourier coeffients?

In [ ]:
# YOUR CODE HERE



In [ ]:
import scipy.signal as ss

s_win = ss.windows.tukey(s.size) * s

plt.figure(figsize=(15, 2))
plt.plot(s_win)

In [ ]:
S_win = np.fft.rfft(s_win)

plt.plot(freq, np.abs(S_win)/n)
plt.xlim(0, 500)

## Filtering

We can apply a frequency filter in the Fourier domain:

### EXERCISE

Make a 1D array with a step function. It shoudl be the same legth as `S_win`. It should have a value of zero up to the 300 Hz point, then a value of 1 above that point.

Plot your function with `S_win`.

In [ ]:
# YOUR CODE HERE



In [ ]:
locut = np.zeros(S_win.size)
locut[freq >= 300] = 1

plt.plot(freq, np.abs(S_win)/n)
plt.plot(freq, locut, c='C2')
plt.fill_between(freq, locut, color='C2', alpha=0.2)
plt.xlim(0, 500)

### EXERCISE

Now try multiplying `S_win` by your step function.

Then pass the result through `np.fft.irfft()` and plot the result. Try listening to it!

In [ ]:
# YOUR CODE HERE



In [ ]:
S_new = S_win * locut

plt.plot(freq, np.abs(S_new)/n)
plt.xlim(0, 500)

In [ ]:
s_new = np.fft.irfft(S_new)

plt.figure(figsize=(15, 3))
plt.plot(s_new)

In [ ]:
Audio(s_new, rate=fs)

In [ ]:
s_new_constructed = ss.windows.tukey(s.size) * (s_[1] + s_[2])

np.allclose(s_new, s_new_constructed, atol=1e-3)

## Convolution

In [ ]:
r = np.random.randn(1000)

plt.plot(r)

In [ ]:
import bruges as bg

w = bg.filters.ricker(1.000, .001, 25)

plt.plot(w)

In [ ]:
r_sm = np.convolve(r, w, mode='same')

plt.plot(r_sm)

Now we compute the convolution as the product of two spectrums:

In [ ]:
R = np.fft.rfft(r)
W = np.fft.rfft(w)

RW = R * W   # This is equivalent to time domain convolution.

rw_ = np.fft.irfft(RW)

rw = np.fft.fftshift(rw_)  # This bit is important — the signal is not in order.

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(rw, lw=5, c='pink', label='fft')
plt.plot(r_sm, label='convolve')
plt.legend()

There's an off-by-one error, not sure where that is coming from.

## More good stuff in `scipy.signal`

The `welch` function is a bit easier to use, but you need to pay attention to its parameters.

In [ ]:
f, Pxx = ss.welch(s, fs=fs, scaling='spectrum')

In [ ]:
plt.plot(f, Pxx)
plt.xlim(0, 500)

## Scale space

There are lots of alternatives to the Fourier transform. One of the most important is the wavelet transform. This casts the data into a 'scale' domain, producing a 'scalogram'. Instead of being monotonic sines and cosines, the components are polytonic 'wavelets' of some kind.

Let's use the Morlet wavelet to decompose `s_new`, our two-tone signal.

In [ ]:
scales = np.arange(1, 25)
cwt = ss.cwt(s, ss.morlet2, scales)

cwt = np.log(abs(cwt.real))

plt.imshow(cwt.real, aspect='auto', origin='lower', vmax=abs(cwt).max(), vmin=-abs(cwt).max())

In [ ]:
cwt.real

## Images and other data

By the way, we can make Fourier transforms of any data — especially regularly sampled data — including well logs, seismic data, and images.

In [ ]:
import segyio

with segyio.open('../data/Penobscot_0-1000ms.sgy') as s:
    seismic = segyio.cube(s)[::2]

In [ ]:
img = seismic[:, :, 90]

plt.figure(figsize=(15, 10))
plt.imshow(img, cmap='gray', aspect='auto')

In [ ]:
import scipy.fft as sf

S = sf.fft2(seismic[:, :, 90])

In [ ]:
S.shape

In [ ]:
plt.figure(figsize=(15, 12))
plt.imshow(np.sqrt(np.abs(sf.fftshift(S))))

## Another signal

Let's make another signal.

### EXERCISE

Can you make a signal with a **transient**? For example, make a 5-second 'background' signal at 410 Hz, then add a shorter, louder 2-second signal at 455 Hz in the middle (i.e. starting at 1.5 seconds).

Try adding some noise to the signal as well, using one of the random number generators in `np.random`.

Plot and listen to your signal.

In [ ]:
# YOUR CODE HERE



In [ ]:
tmax, dt = 5.0, 0.001
fs = int(1 / dt)
n = int(1 + tmax / dt)
t = np.linspace(0.0, tmax, n)

# Create two sin waves.
s1 = np.sin(2*np.pi*410*t)
s2 = 2*np.sin(2*np.pi*455*t)

# Create a transient.
s2 *= np.where((t>1.5)&(t<3.5), 1.0, 0.0)

# Add some noise.
noise = 0.0 * np.random.randn(len(t))

s = s1 + s2 + noise

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(s)

In [ ]:
from IPython.display import Audio

Audio(s, rate=fs*3)

### EXERCISE

Plot the spectrum of your signal.

In [ ]:
# YOUR CODE HERE



Notice that you cannot tell how the two frequencies are related in time.

That's when we need time-frequency representations.

## Time-frequency representations

In [ ]:
import scipy.signal as ss

window = 64
step = 1

f, t, Sxx = ss.spectrogram(s,
                           fs=fs,
                           nperseg=window,
                           noverlap=window - step,
                           scaling='spectrum',
                           mode='magnitude',
                          )

plt.figure(figsize=(12, 6))
plt.imshow(Sxx, origin='lower', extent=(0, t[-1], 0, fs/2), aspect='auto', interpolation='none')
plt.ylim(300, 500)

---

## Spectral decomposition on seismic data

### Check out the [`Spectral_decomposition.ipynb`](./Spectral_decomposition.ipynb) notebook.

---

---

&copy; 2021 Agile Scientific, licenced CC-BY and Apache 2.0